# Problem 1

This is a simulation of Problem 1 from https://www.osti.gov/servlets/purl/1769096

It is a simple trasport problem consisting of a constant absorption by a sphere of a boundary source.

In [1]:
import os
import sys

## Using this Notebook
Before running this example, make sure that the **Python module of OpenSn** was installed.

### Converting and Running this Notebook from the Terminal
To run this notebook from the terminal, simply type:

`jupyter nbconvert --to python --execute problem_one.ipynb`.

To run this notebook in parallel (for example, using 4 processes), simply type:

`mpiexec -n 4 jupyter nbconvert --to python --execute problem_1.ipynb`.

In [2]:
from mpi4py import MPI
size = MPI.COMM_WORLD.size
rank = MPI.COMM_WORLD.rank

if rank == 0:
    print(f"Running the first spherical problem with {size} MPI processors.")

Running the first spherical problem with 1 MPI processors.


## Import Requirements

Import required classes and functions from the Python interface of OpenSn. Make sure that the path
to PyOpenSn is appended to Python's PATH.

In [3]:
# assuming that the execute dir is the notebook dir
# this line is not necessary when PyOpenSn is installed using pip
# sys.path.append("../../../..")

from pyopensn.mesh import OrthogonalMeshGenerator, KBAGraphPartitioner, FromFileMeshGenerator, PETScGraphPartitioner
from pyopensn.xs import MultiGroupXS
from pyopensn.source import VolumetricSource
from pyopensn.aquad import GLCProductQuadrature3DXYZ, GLCProductQuadrature2DRZ
from pyopensn.solver import DiscreteOrdinatesProblem, SteadyStateSolver, DiscreteOrdinatesCurvilinearProblem
from pyopensn.diffusion import DFEMDiffusionSolver, CFEMDiffusionSolver
from pyopensn.fieldfunc import FieldFunctionInterpolationVolume, FieldFunctionGridBased
from pyopensn.context import UseColor, Finalize
from pyopensn.logvol import SphereLogicalVolume, BooleanLogicalVolume, RPPLogicalVolume
from pyopensn.math import Vector3, ScalarSpatialMaterialFunction

OpenSn version 0.0.1
2025-05-10 12:44:02 Running OpenSn with 1 processes.



In [4]:
from scipy.special import exp2
import numpy as np
import math

##### Disable colorized output.

In [5]:
UseColor(False)

### File Mesh Generation

This time the problem is solved using a GMSH mesh and is imported into OpenSn below.

In [6]:
meshgen = FromFileMeshGenerator(
    filename="./vtk_meshes/two_spheres.msh",
    partitioner=PETScGraphPartitioner(type='parmetis'),
)
grid = meshgen.Execute()

# Export
grid.ExportToPVTU("Read_3D_gmsh")

[0]  FromFileMeshGenerator: Generating UnpartitionedMesh
[0]  Making unpartitioned mesh from Gmsh file ./vtk_meshes/two_spheres.msh (format v2.2)
[0]  Mesh identified as 3D.
[0]  Done checking cell-center-to-face orientations
[0]  00:00:02.4 Establishing cell connectivity.
[0]  00:00:02.4 Vertex cell subscriptions complete.
[0]  00:00:02.5 Surpassing cell 2014 of 20138 (10%)
[0]  00:00:02.6 Surpassing cell 4028 of 20138 (20%)
[0]  00:00:02.6 Surpassing cell 6042 of 20138 (30%)
[0]  00:00:02.7 Surpassing cell 8056 of 20138 (40%)
[0]  00:00:02.7 Surpassing cell 10069 of 20138 (50%)
[0]  00:00:02.8 Surpassing cell 12083 of 20138 (60%)
[0]  00:00:02.8 Surpassing cell 14097 of 20138 (70%)
[0]  00:00:02.8 Surpassing cell 16111 of 20138 (80%)
[0]  00:00:02.8 Surpassing cell 18125 of 20138 (90%)
[0]  00:00:02.9 Surpassing cell 20138 of 20138 (100%)
[0]  00:00:02.9 Establishing cell boundary connectivity.
[0]  00:00:02.9 Done establishing cell connectivity.
[0]  Done processing ./vtk_meshes/two

In [7]:
grid.SetUniformBlockID(1)

[0]  00:00:03.8 Done setting block id 1 to all cells


In [18]:
def mat_id_function(pt, cur_id):
    if ((pt.x ** 2 + pt.y ** 2 + pt.z ** 2) ** (1/2)) < 0.5:
        return 0
    return cur_id

In [19]:
grid.SetBlockIDFromFunction(mat_id_function)

In [7]:
D = [0.,100000., 100000.]
Q = [0.,1.0, 0.]
XSa = [0.,0., 0.]

def D_coef(i, pt):
    return D[i]

def Q_ext(i, pt):
    return Q[i]

def Sigma_a(i, pt):
    return XSa[i]

In [8]:
d_coef_fn = ScalarSpatialMaterialFunction(D_coef)
Q_ext_fn = ScalarSpatialMaterialFunction(Q_ext)
Sigma_a_fn = ScalarSpatialMaterialFunction(Sigma_a)

In [9]:
sphere_vol = SphereLogicalVolume(r=0.5)

sphere_bndry = "0"

grid.SetBoundaryIDFromLogicalVolume(sphere_vol, sphere_bndry, True)

outer_vol = SphereLogicalVolume()

outer_bndry = "1"

grid.SetBoundaryIDFromLogicalVolume(outer_vol, outer_bndry, True)

In [10]:
phys = DFEMDiffusionSolver(
    name="problem_one",
    mesh=grid,
)
phys.SetOptions(boundary_conditions=[
    {"boundary": sphere_bndry, "type": "dirichlet", "coeffs":[0.5]},
    {"boundary": outer_bndry, "type": "vacuum"}
    
])
phys.SetDCoefFunction(d_coef_fn)
phys.SetQExtFunction(Q_ext_fn)
phys.SetSigmaAFunction(Sigma_a_fn)

[0]  Boundary 0 set as Dirichlet with value 0.5
[0]  Boundary 1 set as Vacuum.


In [11]:
phys.Initialize()
phys.Execute()

[0]  
[0]  00:00:16.2 problem_one: Initializing DFEM Diffusion solver 
[0]  Global num cells: 20138
[0]  Boundary 1 set to robin.0.25,0.5,0
[0]  Num local DOFs: 80552
[0]  Num globl DOFs: 80552
[0]  
[0]  Executing DFEM IP Diffusion solver
[0]  Assembling system: 
[0]  Global assembly
[0]  Done global assembly
[0]  Solving: 
[0]  problem_one iteration    0 - Residual 4.3443229e+03
[0]  problem_one iteration    1 - Residual 1.0062408e-03
[0]  Done solving


In [12]:
fflist = phys.GetFieldFunctions()
vtk_basename = "problem_one"
FieldFunctionGridBased.ExportMultipleToVTK(
    [fflist[0]],  # export only the flux of group 0 (first []), moment 0 (second [])
    vtk_basename
)

[0]  Exporting field functions to VTK with file base "problem_one"
[0]  Done exporting field functions to VTK.


In [13]:
def average_vol(vol0, r1, r2):
    ffvol = FieldFunctionInterpolationVolume()
    ffvol.SetOperationType("avg")
    ffvol.SetLogicalVolume(vol0)
    ffvol.AddFieldFunction(fflist[0])
    ffvol.Initialize()
    ffvol.Execute()
    avgval = ffvol.GetValue()
    print("Radius: {:.2f} {:.2f} {:.6f}".format(r1, r2, avgval))
    return avgval

def create_vols(N_vols, rmax):
    r_vals = np.linspace(0, rmax, N_vols + 1)
    vols = np.empty(N_vols)
    avgphi = np.zeros(N_vols)
    for i in range(N_vols):
        if i != 0:
            inner_vol = SphereLogicalVolume(r=r_vals[i])
            outer_vol = SphereLogicalVolume(r=r_vals[i + 1])
            vol = BooleanLogicalVolume(parts=[{"op":True,"lv":outer_vol},{"op":False,"lv":inner_vol}])
        else:
            vol = SphereLogicalVolume(r=r_vals[i + 1])
        avgphi[i] = average_vol(vol, r_vals[i], r_vals[i+1])
    return avgphi

In [14]:
n_vols = 10
sim_vals = create_vols(n_vols, 1)

Radius: 0.00 0.10 0.082297
Radius: 0.10 0.20 0.082297
Radius: 0.20 0.30 0.082297
Radius: 0.30 0.40 0.082297
Radius: 0.40 0.50 0.082297
Radius: 0.50 0.60 0.082297
Radius: 0.60 0.70 0.082297
Radius: 0.70 0.80 0.082297
Radius: 0.80 0.90 0.082297
Radius: 0.90 1.00 0.082297


In [15]:
def get_phi(r, q, a):
    phi = q*(a+(a**2 - r**2)/(2*r)*math.log((a+r)/(abs(r-a)), 10))
    return phi              

In [16]:
q = 1/(2*math.pi)
a = 0.5
r_vals = np.linspace(0.05, 0.95, n_vols)
phi = np.zeros(n_vols)
for i in range(n_vols):
    phi[i] = get_phi(r_vals[i], q, a)
    print(r_vals[i], phi[i])

0.05 0.11390666515699852
0.15 0.11202508399960438
0.25 0.10805354884951328
0.35 0.10141563910932869
0.44999999999999996 0.09031880258810289
0.5499999999999999 0.06953385229872941
0.65 0.06089578655122078
0.75 0.05640152753281415
0.85 0.053643589744471414
0.95 0.05180313667985476


In [17]:
err = np.zeros(n_vols)
err[:] = abs(phi[:]-sim_vals[:])/phi[:]
print(err)

[0.27750871 0.26537372 0.2383724  0.18852202 0.08882152 0.18354639
 0.35143231 0.45911879 0.5341353  0.58863973]


## Finalize (for Jupyter Notebook only)

In Python script mode, PyOpenSn automatically handles environment termination. However, this
automatic finalization does not occur when running in a Jupyter notebook, so explicit finalization
of the environment at the end of the notebook is required. Do not call the finalization in Python
script mode, or in console mode.

Note that PyOpenSn's finalization must be called before MPI's finalization.


In [52]:
from IPython import get_ipython

def finalize_env():
    Finalize()
    MPI.Finalize()

ipython_instance = get_ipython()
if ipython_instance is not None:
    ipython_instance.events.register("post_execute", finalize_env)


Elapsed execution time: 00:19:26.3
2025-05-10 12:43:50 OpenSn finished execution.
